# [NUTS statistical regions of Italy](https://en.wikipedia.org/wiki/NUTS_statistical_regions_of_Italy)

## Importing libraries

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from time import sleep

import requests
from bs4 import BeautifulSoup as sp

## Fetching the table and links

In [2]:
url = 'https://en.wikipedia.org/wiki/NUTS_statistical_regions_of_Italy'

In [3]:
r = requests.get(url)
r

<Response [200]>

In [4]:
soup = sp(r.content, 'lxml')

In [5]:
table = soup.find('div', {'class' : 'mw-parser-output'}).find_all('table')[1]

In [6]:
df = pd.DataFrame(columns = ['NUTS_1', 'Code_1', 'NUTS_2', 'Code_2', 'NUTS_3', 'Code_3', 'NUTS_2_Link', 'NUTS_3_Link'])

for idx, rows in enumerate(table.find_all('tr')):
    if idx != 0:
        if len(rows.find_all('td')) == 6:
            tetx_list, links_list = [], []
            
            for txt in rows.find_all('td'):
                tetx_list.append(txt.text.strip())
            for links in rows.find_all('a'):
                links_list.append(links['href'])
            tetx_list.extend([links_list[1], links_list[-2]])
            df.loc[len(df)] = tetx_list
            
        if len(rows.find_all('td')) == 2:
            tetx_list, links_list = [np.nan, np.nan, np.nan, np.nan], []
            
            for txt in rows.find_all('td'):
                tetx_list.append(txt.text.strip())
            for links in rows.find_all('a'):
                links_list.append(links['href'])
            tetx_list.extend([np.nan, links_list[0]])
            df.loc[len(df)] = tetx_list
            
        if len(rows.find_all('td')) == 4:
            tetx_list, links_list = [np.nan, np.nan], []
            
            for txt in rows.find_all('td'):
                tetx_list.append(txt.text.strip())
            for links in rows.find_all('a'):
                links_list.append(links['href'])
            tetx_list.extend([links_list[0], links_list[-2]])
            df.loc[len(df)] = tetx_list

df.head(5)

,NUTS_1,Code_1,NUTS_2,Code_2,NUTS_3,Code_3,NUTS_2_Link,NUTS_3_Link
0,Northwest Italy,ITC,Piemonte,ITC1,Torino,ITC11,/wiki/File:Flag_of_Piedmont.svg,/wiki/File:Flag_of_the_Province_of_Turin.svg
1,NaN,NaN,NaN,NaN,Vercelli,ITC12,NaN,/wiki/File:Image_unavailable.png
2,NaN,NaN,NaN,NaN,Biella,ITC13,NaN,/wiki/File:Image_unavailable.png
3,NaN,NaN,NaN,NaN,Verbano-Cusio-Ossola,ITC14,NaN,/wiki/File:Flag_of_the_Province_of_Verbano-Cus...
4,NaN,NaN,NaN,NaN,Novara,ITC15,NaN,/wiki/File:Flag_of_the_Province_of_Novara.gif


In [7]:
df = df.ffill().copy()

BASE_URL = 'https://en.wikipedia.org/'
df.NUTS_2_Link = BASE_URL + df.NUTS_2_Link.map(str)
df.NUTS_3_Link = BASE_URL + df.NUTS_3_Link.map(str)

df.head()

,NUTS_1,Code_1,NUTS_2,Code_2,NUTS_3,Code_3,NUTS_2_Link,NUTS_3_Link
0,Northwest Italy,ITC,Piemonte,ITC1,Torino,ITC11,https://en.wikipedia.org//wiki/File:Flag_of_Pi...,https://en.wikipedia.org//wiki/File:Flag_of_th...
1,Northwest Italy,ITC,Piemonte,ITC1,Vercelli,ITC12,https://en.wikipedia.org//wiki/File:Flag_of_Pi...,https://en.wikipedia.org//wiki/File:Image_unav...
2,Northwest Italy,ITC,Piemonte,ITC1,Biella,ITC13,https://en.wikipedia.org//wiki/File:Flag_of_Pi...,https://en.wikipedia.org//wiki/File:Image_unav...
3,Northwest Italy,ITC,Piemonte,ITC1,Verbano-Cusio-Ossola,ITC14,https://en.wikipedia.org//wiki/File:Flag_of_Pi...,https://en.wikipedia.org//wiki/File:Flag_of_th...
4,Northwest Italy,ITC,Piemonte,ITC1,Novara,ITC15,https://en.wikipedia.org//wiki/File:Flag_of_Pi...,https://en.wikipedia.org//wiki/File:Flag_of_th...


## Fetching downloadable image urls

In [8]:
NUTS_2_df = pd.DataFrame(columns = ['NUTS_2_Link', 'NUTS_2_Images'])
for link in tqdm(df.NUTS_2_Link.unique()):
    r = requests.get(link)
    soup = sp(r.content, 'lxml')
    try:
        img_link  = 'https:' + soup.find('div', {'class' : 'fullImageLink'}).find('a')['href']
    except:
        img_link = np.nan
    NUTS_2_df.loc[len(NUTS_2_df)] = [link, img_link]
    sleep(3)

100%|███████████████████████████████████████████| 20/20 [01:08<00:00,  3.41s/it]


In [9]:
NUTS_3_df = pd.DataFrame(columns = ['NUTS_3_Link', 'NUTS_3_Images'])
for link in tqdm(df.NUTS_3_Link.unique()):
    r = requests.get(link)
    soup = sp(r.content, 'lxml')
    try:
        img_link  = 'https:' + soup.find('div', {'class' : 'fullImageLink'}).find('a')['href']
    except:
        img_link = np.nan
    NUTS_3_df.loc[len(NUTS_3_df)] = [link, img_link]
    sleep(3)

100%|███████████████████████████████████████████| 94/94 [05:22<00:00,  3.43s/it]


## Merging image links to the table dataframe

In [10]:
df = df.merge(NUTS_2_df, how = 'right').copy()

In [11]:
df = df.merge(NUTS_3_df, how = 'right').copy()

## Filling missing values manually

In [12]:
df[df.isnull().any(axis = 1)]

,NUTS_1,Code_1,NUTS_2,Code_2,NUTS_3,Code_3,NUTS_2_Link,NUTS_3_Link,NUTS_2_Images,NUTS_3_Images
71,Northeast Italy,ITH,Veneto,ITH3,Belluno,ITH33,https://en.wikipedia.org//wiki/File:Flag_of_Ve...,https://en.wikipedia.org//wiki/Province_of_Bel...,https://upload.wikimedia.org/wikipedia/commons...,NaN
79,Northeast Italy,ITH,Friuli-Venezia Giulia,ITH4,Trieste,ITH44,https://en.wikipedia.org//wiki/File:Flag_of_Fr...,https://en.wikipedia.org//wiki/Province_of_Tri...,https://upload.wikimedia.org/wikipedia/commons...,NaN


In [13]:
df.loc[71].NUTS_3_Images = 'https://upload.wikimedia.org/wikipedia/en/6/65/Provincia_di_Belluno-Stemma.png'

In [14]:
df[df.isnull().any(axis = 1)]

,NUTS_1,Code_1,NUTS_2,Code_2,NUTS_3,Code_3,NUTS_2_Link,NUTS_3_Link,NUTS_2_Images,NUTS_3_Images
79,Northeast Italy,ITH,Friuli-Venezia Giulia,ITH4,Trieste,ITH44,https://en.wikipedia.org//wiki/File:Flag_of_Fr...,https://en.wikipedia.org//wiki/Province_of_Tri...,https://upload.wikimedia.org/wikipedia/commons...,NaN


## Saving to CSV file

In [15]:
df.to_csv('NUTS_Code_RAW.csv', index = False)
df[['NUTS_1', 'Code_1', 'NUTS_2', 'Code_2', 'NUTS_3', 'Code_3', 'NUTS_2_Images', 'NUTS_3_Images']].to_csv('NUTS_Code_Processed.csv', index = False)

# Contributer info

<p><strong>Jishnu S G</strong></p>

<ul>
  <li><p><a href = 'https://linkedin.com/in/jishnukoliyadan/'>Linkedin</a></p></li>
  <li><p><a href = 'https://github.com/jishnukoliyadan'>GitHub</a></p></li>
</ul>  

<p>
    <a href = 'https://www.buymeacoffee.com/jishnukoliyadan'>
    <img src = 'https://www.buymeacoffee.com/assets/img/guidelines/download-assets-sm-1.svg' alt = 'https://www.buymeacoffee.com/jishnukoliyadan' width = 11%></a>
</p>